In [ ]:
from utils import get_EcStEnd_list
import pandas as pd

In [ ]:
def get_power_consumption_factor(a,b):
    '''
    return -ve values
    '''
    if a=='Imaging' :
        return 
    elif a=='Readout' & b == 0:
        return
    elif a=='Readout' & b == 1:
        return 
    elif a=='downlink_from_camera' & b == 0:
        return
    elif a=='downlink_from_camera' & b == 1:
        return
    elif a=='no_i_no_g' & b == 0:
        return
    elif a=='no_i_no_g' & b == 1:
        return
    
def get_power_recieve_factor(a,b):
    if a=='Imaging' :
        return 
    elif a=='Readout' & b == 0:
        return
    elif a=='Readout' & b == 1:
        return
    elif a=='downlink_from_camera' & b == 0:
        return
    elif a=='downlink_from_camera' & b == 1:
        return
    elif a=='no_i_no_g' & b == 0:
        return
    elif a=='no_i_no_g' & b == 1:
        return
 

In [3]:

def camera_memory_estimator(s,image_capture_planned_df,current_time,imaging_rate,readout_rate,initial_memory):

    thisSatId_df = image_capture_planned_df[image_capture_planned_df['SatID']==s]
    thisSatId_df = thisSatId_df[thisSatId_df['start_time']>=current_time]
    thisSatId_df.sort_values(by='start_time',inplace = True)
    thisSatId_df['delta_memory_till_now'] = thisSatId_df[['start_time','end_time','operation']]\
                .apply(lambda a: (a['end_time']-a['start_time'])*imaging_rate \
                if a['operation']=='Imaging' \
                else (a['end_time']-a['start_time'])*readout_rate*-1)
    
    #thisSatId_df['end_time'] -  thisSatId_df['start_time']*imaging_rate
    return thisSatId_df['delta_memory_till_now'].sum() + initial_memory

def readout_memory_estimator(s,image_capture_planned_df,current_time,readout_rate,initial_memory):
    thisSatId_df = image_capture_planned_df[image_capture_planned_df['SatID']==s]
    thisSatId_df = thisSatId_df[thisSatId_df['start_time']>=current_time]
    thisSatId_df = thisSatId_df[thisSatId_df['operation'].isin(['Readout','downlink_from_camera'])]
    thisSatId_df.sort_values(by='start_time',inplace = True)

    thisSatId_df['delta_memory_till_now'] = thisSatId_df[['start_time','end_time','operation']]\
                .apply(lambda a: (a['end_time']-a['start_time'])*readout_rate \
                if a['operation']=='Readout' \
                else 0)
    
    #thisSatId_df['end_time'] -  thisSatId_df['start_time']*imaging_rate
    return thisSatId_df['delta_memory_till_now'].sum() + initial_memory

def power_estimator(s,image_capture_planned_df,eclipse_df,current_time,initial_power,max_power):


    thisSatId_df = image_capture_planned_df[image_capture_planned_df['SatID']==s]
    thisSatId_df = thisSatId_df[thisSatId_df['start_time'] >= current_time]

    thisSatId_df.sort_values(by='start_time',inplace = True)

    thisSatId_df['till_now_max'] = thisSatId_df.groupby('SatID')['end_time'].cummax()
    thisSatId_df['prev_max'] = thisSatId_df.groupby('SatID')['till_now_max'].shift(1)

    imgGS_union_df1 = thisSatId_df[thisSatId_df['start_time'] > thisSatId_df['prev_max'] + 1] 
    imgGS_union_df1['start_time1'] = imgGS_union_df1['prev_max'] + 1 #TODO1 +1 is okay ?
    imgGS_union_df1['end_time1'] = imgGS_union_df1['start_time'] - 1
    imgGS_union_df1['encoded_stripId'] = 'no_i_no_g'
    imgGS_union_df1['gsID'] = 'no_i_no_g'

    imgGS_union_df1 = imgGS_union_df1.drop(['start_time', 'end_time','till_now_max','prev_max'], axis=1)
    imgGS_union_df1.rename(columns={'start_time1':'start_time','end_time1':'end_time'},inplace=True)
    #imgGS_union_df1 ==> contains TW without img and without gs pass  table without eclipse divide
    thisSatId_df = pd.concat([thisSatId_df,imgGS_union_df1])

    thisSatId_df['EcStEnd_list'] = thisSatId_df[['SatID','start_time','end_time']].\
                                          apply( lambda a : \
                                          get_EcStEnd_list(a['start_time'],a['end_time'],df = eclipse_df ),axis= 1 )
    
    thisSatId_df['len_EcStEnd_list'] = thisSatId_df['EcStEnd_list'].apply(lambda a : len(a))
    print('len_before_eclipse_transition_divide=',len(thisSatId_df))
    thisSatId_df = thisSatId_df.explode('EcStEnd_list')
    print('len_after_eclipse_transition_divide=',len(thisSatId_df))
    thisSatId_df['new_eclipse'] = thisSatId_df['EcStEnd_list'].apply(lambda a : a[0])
    thisSatId_df['new_start_time'] = thisSatId_df['EcStEnd_list'].apply(lambda a : a[1])
    thisSatId_df['new_end_time'] = thisSatId_df['EcStEnd_list'].apply(lambda a : a[2])
    thisSatId_df.drop(['start_time','end_time','Eclipse','EcStEnd_list'],axis=1,inplace=True)
    thisSatId_df.rename(columns={'new_start_time':'start_time','new_end_time':'end_time','new_eclipse':'Eclipse'},inplace=True)    
    thisSatId_df = thisSatId_df.drop(['till_now_max','prev_max'], axis=1)

    thisSatId_df['power_consumption_factor'] = thisSatId_df[['operation','Eclipse']].apply(lambda a: get_power_consumption_factor(a['operation'],a['Eclipse']),axis =  1) 
    thisSatId_df['power_recieve_factor'] = thisSatId_df[['operation','Eclipse']].apply(lambda a: get_power_recieve_factor(a['operation'],a['Eclipse']),axis =  1) 

    thisSatId_df['power_gain'] = (thisSatId_df['end_time'] - thisSatId_df['start_time'])*thisSatId_df['power_recieve_factor']
    thisSatId_df['power_consumption'] = (thisSatId_df['end_time'] - thisSatId_df['start_time'])*thisSatId_df['power_consumption_factor']
    thisSatId_df['delta_power_gain'] = thisSatId_df['power_consumption'] + thisSatId_df['power_consumption'] 
    thisSatId_df['delta_power_gain'] = thisSatId_df['delta_power_gain'].apply(lambda a : max_power if a>= max_power else a )

    for delta in list(thisSatId_df['delta_power_gain']):
        initial_power = initial_power + delta
        if initial_power >= max_power:
            initial_power = max_power

    return initial_power
            
def camera_detector_thermal_estimator(s,image_capture_planned_df,current_time):
    ''' 
    
    '''

    thisSatId_df = image_capture_planned_df[image_capture_planned_df['SatID']==s]
    thisSatId_df = thisSatId_df[thisSatId_df['start_time']>=current_time]
    thisSatId_df = thisSatId_df[thisSatId_df['operation'].isin(['Imaging'])]

    thisSatId_df.sort_values(by='start_time',inplace=True)
    thisSatId_df['till_now_max'] = thisSatId_df.groupby('SatID')['end_time'].cummax()
    thisSatId_df['prev_max'] = thisSatId_df.groupby('SatID')['till_now_max'].shift(1)

    thisSatId_df1 = thisSatId_df[thisSatId_df['start_time'] > thisSatId_df['prev_max'] + 1] 
    thisSatId_df1['start_time1'] = thisSatId_df1['prev_max'] + 1 #TODO1 +1 is okay ?
    thisSatId_df1['end_time1'] = thisSatId_df1['start_time'] - 1
    thisSatId_df1['encoded_stripId'] = 'no_i_no_g'
    thisSatId_df1['gsID'] = 'no_i_no_g'

    thisSatId_df1 = thisSatId_df1.drop(['start_time', 'end_time','till_now_max','prev_max'], axis=1)
    thisSatId_df1.rename(columns={'start_time1':'start_time','end_time1':'end_time'},inplace=True)
    #imgGS_union_df1 ==> contains TW without img and without gs pass  table without eclipse divide
    all_TW_df = pd.concat([thisSatId_df,thisSatId_df1])
    all_TW_df.sort_values(by='start_time',inplace=True)
    




    pass


